In [1]:
from array import array
import math

In [2]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

#### A 2-dimensional vector class

In [3]:
>>> v1 = Vector2d(3, 4)
>>> print(v1.x, v1.y)

3.0 4.0


In [4]:
>>> x, y = v1
>>> x, y

(3.0, 4.0)

In [5]:
>>> v1

Vector2d(3.0, 4.0)

In [6]:
>>> v1_clone = eval(repr(v1))
>>> v1 == v1_clone

True

In [7]:
>>> print(v1)

(3.0, 4.0)


In [8]:
>>> octets = bytes(v1)
>>> octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [9]:
>>> abs(v1)

5.0

In [10]:
>>> bool(v1), bool(Vector2d(0, 0))

(True, False)

#### Test of ``.frombytes()`` class method:

In [11]:
>>> v1_clone = Vector2d.frombytes(bytes(v1))
>>> v1_clone

Vector2d(3.0, 4.0)

In [12]:
>>> v1 == v1_clone

True

#### Tests of ``format()`` with Cartesian coordinates:

In [13]:
>>> format(v1)

'(3.0, 4.0)'

In [14]:
>>> format(v1, '.2f')

'(3.00, 4.00)'

In [15]:
>>> format(v1, '.3e')

'(3.000e+00, 4.000e+00)'

#### Tests of the ``angle`` method::

In [16]:
>>> Vector2d(0, 0).angle()

0.0

In [17]:
>>> Vector2d(1, 0).angle()

0.0

In [18]:
>>> epsilon = 10**-8
>>> abs(Vector2d(0, 1).angle() - math.pi/2) < epsilon

True

In [19]:
>>> abs(Vector2d(1, 1).angle() - math.pi/4) < epsilon

True

#### Tests of ``format()`` with polar coordinates:

In [20]:
>>> format(Vector2d(1, 1), 'p')  # doctest:+ELLIPSIS

'<1.4142135623730951, 0.7853981633974483>'

In [21]:
>>> format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [22]:
>>> format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

#### Tests of `x` and `y` read-only properties:

In [23]:
>>> v1.x, v1.y

(3.0, 4.0)

In [24]:
>>> v1.x = 123

AttributeError: can't set attribute 'x'

#### Tests of hashing:

In [25]:
>>> v1 = Vector2d(3, 4)
>>> v2 = Vector2d(3.1, 4.2)
>>> hash(v1), hash(v2)

(7, 384307168202284039)

In [26]:
>>> len(set([v1, v2]))

2